In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'jax'

In [ ]:
# Define the polynomial function with given coefficients
def Optimalpolynomial(x):
    return 2*x**5 - 3*x**4 + x**3 - 5*x**2 + 4*x - 1


# Generate data points
xx = jnp.linspace(-6, 6, 100)
data_points = list(zip(xx, Optimalpolynomial(xx)))

x_values = jnp.array([x for x, y in data_points])
y_values = jnp.array([y for x, y in data_points])


# Plot the original function
plt.scatter(x_values, y_values)
plt.title("Sampled function that needs to be approximated")
plt.show()

# Define the polynomial curve to be fitted
def Curve(x, arr):
    return arr[0] + arr[1]*x + arr[2]*x**2 + arr[3]*x**3 + arr[4]*x**4 + arr[5]*x**5

#Huber Loss Function
def Loss(arr, delta=1.0):
    predictions = Curve(x_values, arr)
    error = predictions - y_values
    is_small_error = jnp.abs(error) <= delta
    squared_loss = jnp.square(error) / 2
    linear_loss = delta * (jnp.abs(error) - delta / 2)
    return jnp.mean(jnp.where(is_small_error, squared_loss, linear_loss))

# Ideal loss with optimal parameters
IdealLoss = Loss(jnp.array([-1, 4, -5, 1, -3, 2]))
IdealWeights = [-1, 4, -5, 1, -3, 2]
print(f'Ideal Loss: {IdealLoss}')

In [ ]:
shift = - 5.0
weights = jnp.array([IdealWeights[0]+shift , IdealWeights[1]+shift ,IdealWeights[2]+shift , IdealWeights[3]+shift , IdealWeights[4]+shift , IdealWeights[5]+shift ])

#weights = jnp.array([np.random.normal() * 2.0 for _ in range(6)])

# initializing weights close to ideal weights that we want to predict


# Define gradient descent step
def gradient_descent(weights, gradient, learning_rate, clip_value):
    grad_clipped = jnp.clip(gradient, -clip_value, clip_value)  # Limit gradient magnitude
    return weights - learning_rate * grad_clipped

# Gradient of the loss function
gradient = jax.grad(Loss)

# Gradient descent parameters
tolerance = 1e-10
max_iteration = 1000
learning_rate = 1e-2   # 1e-2 is getting us good results
clip_value = 1.0

grad = gradient(weights)

loss_values = [Loss(weights)]

iteration = 0
#Gradient descent loop
while jnp.linalg.norm(grad) >= tolerance and (iteration <= max_iteration):
    weights = gradient_descent(weights, grad, learning_rate, clip_value)
    grad = gradient(weights)
    loss = Loss(weights)
    loss_values.append(loss)

    iteration += 1




print(f'Optimized weights: {weights}')
print(f"Ideal weights: {IdealWeights}")
print(f'Optimized Loss: {Loss(weights)}')
# Plot the fitted curve along with the original data points
plt.plot(x_values, Curve(x_values, weights), label="Predicted", color='red')
plt.scatter(x_values, y_values, label="Actual", color='blue')
plt.xlim(-4,+4)
plt.ylim(-9,+9)
plt.legend()
plt.title("Fitted curve vs Actual data points")
plt.show()

print("\n")

plt.plot(loss_values)
plt.title("Loss over iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()